In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter

In [2]:
def check_kw_freq(df, kws: list, op = 'or', attr = 'JOB_DESCRIPTION', verbose = False):
    
    kw_booltable = df[attr].str.contains(kws[0])
    for i in range(1, len(kws)):
        if op == 'or':
            kw_booltable =   kw_booltable | df[attr].str.contains(kws[i])
        else:
            kw_booltable =   kw_booltable & df[attr].str.contains(kws[i])
    
    if verbose:
        return kw_booltable
    else:
        return kw_booltable.sum()

In [3]:
postings_data = pd.read_csv('../social_datathon_0/UK_Postings_Sample.csv', low_memory=False)

In [4]:
postings_data.columns

Index([' ', 'POSTED', 'TITLE_NAME', 'SOC_4', 'SOC_4_NAME', 'SOC_3',
       'SOC_3_NAME', 'SOC_2', 'SOC_2_NAME', 'LOT_OCCUPATION_NAME',
       'LOT_CAREER_AREA_NAME', 'CITY_NAME', 'NUTS1_NAME', 'NUTS3_NAME',
       'TTWA_NAME', 'COUNTRY_NAME', 'COMPANY_NAME', 'COMPANY_IS_STAFFING',
       'UK_SIC_2', 'INDUSTRY', 'IS_INTERNSHIP', 'IS_REMOTE',
       'SPECIALIZED_SKILLS_NAME', 'COMMON_SKILLS_NAME', 'SOFTWARE_SKILLS_NAME',
       'CERTIFICATIONS_NAME', 'EDULEVELS_NAME_MIN', 'SALARY',
       'JOB_DESCRIPTION'],
      dtype='object')

In [8]:
N_jobs, N_attr = postings_data.shape
print(N_jobs)

100011


In [9]:
for sk in ['SPECIALIZED_SKILLS_NAME', 'COMMON_SKILLS_NAME','SOFTWARE_SKILLS_NAME' ]:
    postings_data[sk] = postings_data[sk].astype(str)

postings_data['POSTED'] = pd.to_datetime(postings_data['POSTED'])
postings_data['JOB_DESCRIPTION'] = postings_data['JOB_DESCRIPTION'].str.lower()

In [10]:
postings_data.head()

,,POSTED,TITLE_NAME,SOC_4,SOC_4_NAME,SOC_3,SOC_3_NAME,SOC_2,SOC_2_NAME,LOT_OCCUPATION_NAME,...,INDUSTRY,IS_INTERNSHIP,IS_REMOTE,SPECIALIZED_SKILLS_NAME,COMMON_SKILLS_NAME,SOFTWARE_SKILLS_NAME,CERTIFICATIONS_NAME,EDULEVELS_NAME_MIN,SALARY,JOB_DESCRIPTION
0,820881164.0,2021-08-14,Quality Control Laboratory Supervisors,2111.0,Chemical scientists,211.0,Natural and Social Science Professionals,21.0,"Science, Research, Engineering and Technology ...",NaN,...,Scientific research and development,False,False,"[\n ""High-Performance Liquid Chromatography"",...","[\n ""Management""\n]",[],[],Bachelor's or equivalent,NaN,hplc laboratory supervisor\n\nposted by adzuna...
1,820921812.0,2021-08-14,Education and Government Financial Engagement ...,4159.0,Other administrative occupations n.e.c.,415.0,Other Administrative Occupations,41.0,Administrative Occupations,Office / Administrative Assistant,...,Information service activities,False,False,"[\n ""Target Audience"",\n ""Procurement"",\n ""...","[\n ""Ability To Meet Deadlines"",\n ""Influenc...",[],[],NaN,NaN,government engagement manager\nchiswick\n\ngre...
2,820917873.0,2021-08-14,Permanency Specialists,3513.0,Ship and hovercraft officers,351.0,Transport Associate Professionals,35.0,Business and Public Service Associate Professi...,Ship / Boat Captain,...,Air transport,False,False,"[\n ""Basic Safety Training""\n]","[\n ""Management"",\n ""Consulting""\n]",[],"[\n ""Standards Of Training Certification and ...",NaN,NaN,master (tug) - uk - permanent\n\nposted today ...
3,820965527.0,2021-08-15,Technical Services Administrators,5223.0,Metal working production and maintenance fitters,522.0,"Metal Machining, Fitting and Instrument Making...",52.0,"Skilled Metal, Electrical and Electronic Trades",Repair / Service Technician,...,Food and beverage service activities,False,False,"[\n ""IBM Maximo"",\n ""Statistical Analysis"",\...","[\n ""Customer Service"",\n ""Confident Communi...","[\n ""IBM Maximo""\n]",[],NaN,22500.0,estates & technical services administrator\n\n...
4,820991372.0,2021-08-15,Deputy Team Leaders,4123.0,Bank and post office clerks,412.0,Administrative Occupations: Finance,41.0,Administrative Occupations,Project Manager,...,"Financial service activities, except insurance...",False,False,"[\n ""Health Assessment"",\n ""Auditing""\n]","[\n ""Communications"",\n ""Management"",\n ""Mi...","[\n ""Microsoft Excel""\n]",[],NaN,37988.0,independent audit deputy team leader - (dac)\n...


In [11]:
postings_data['EDULEVELS_NAME_MIN'].isnull().sum()/N_jobs

0.8003019667836538

In [12]:
print(postings_data.loc[0]['JOB_DESCRIPTION'])

hplc laboratory supervisor

posted by adzuna on 11/08/2021

postcode
south west
gloucestershire
gloucester
chipping campden - gl55

company name  adzuna
category  scientific
contract  permanent
full/part time  full time
salary  £ up to £30000 per annum

description

nscg are working in partnership with campden bri in the recruitment of a hplc supervisor. having been established for over 100 years, campden bri is well known for its breadth of services. with over 2,400 member companies, including the top 10 uk retailers and the top 15 global food & beverage manufacturers, clients view the organisation as an extension of their own in-house technical team. reporting to the chromatography section manager, the hplc supervisor will undertake the analysis of samples in the hplc laboratory and all associated activities. management of the of the activities and operation of the hplc group. the role is primarily involved with the analysis of foods (and other matrices as required) by hplc, but can 

In [13]:
postings_data[postings_data['JOB_DESCRIPTION'].isnull()]

,,POSTED,TITLE_NAME,SOC_4,SOC_4_NAME,SOC_3,SOC_3_NAME,SOC_2,SOC_2_NAME,LOT_OCCUPATION_NAME,...,INDUSTRY,IS_INTERNSHIP,IS_REMOTE,SPECIALIZED_SKILLS_NAME,COMMON_SKILLS_NAME,SOFTWARE_SKILLS_NAME,CERTIFICATIONS_NAME,EDULEVELS_NAME_MIN,SALARY,JOB_DESCRIPTION
2299,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
2308,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
2322,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
2712,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
5448,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
5922,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
5953,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
29404,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
32271,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN
50789,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN


In [14]:
kws_list = [['green'], ['climate', 'climate-change', 'global warming'], ['renewable', 'renewables'], 
            ['renewable', 'wind power', 'wind energy', 'wind turbine', 'wind turbines','solar power', 'solar energy', 'solar panels','solar panel', 'geothermal power', 'geothermal energy', 'tidal power', 'tidal energy'], 
            ['battery', 'batteries'], ['sustainable', 'sustainability'], ['carbon', 'low-carbon', 'carbon-neutral'], ['greenhouse', 'green house']]
kws_freq = [check_kw_freq(postings_data, kws) for kws in kws_list]

In [15]:
for idx, kws in enumerate(kws_list):
    print(kws, kws_freq[idx])

['green'] 2764
['climate', 'climate-change', 'global warming'] 592
['renewable', 'renewables'] 659
['renewable', 'wind power', 'wind energy', 'wind turbine', 'wind turbines', 'solar power', 'solar energy', 'solar panels', 'solar panel', 'geothermal power', 'geothermal energy', 'tidal power', 'tidal energy'] 704
['battery', 'batteries'] 218
['sustainable', 'sustainability'] 3103
['carbon', 'low-carbon', 'carbon-neutral'] 762
['greenhouse', 'green house'] 48


In [16]:
N_companies = postings_data['COMPANY_NAME'].nunique()
print(N_companies)

15982


In [98]:
spec_skills = postings_data['SPECIALIZED_SKILLS_NAME']
comm_skills = postings_data['COMMON_SKILLS_NAME']
sw_skills = postings_data['SOFTWARE_SKILLS_NAME']

In [102]:
spec_skill_split = [s[4:-2].replace('"', '').split(',\n  ') for s in spec_skills if len(s) > 3 ]
comm_skill_split = [s[4:-2].replace('"', '').split(',\n  ') for s in comm_skills if len(s) > 3 ]
sw_skill_split = [s[4:-2].replace('"', '').split(',\n  ') for s in sw_skills if len(s) > 3 ]

spec_skill_counts = Counter([sk for joblst in spec_skill_split for sk in joblst])
comm_skill_counts = Counter([sk for joblst in comm_skill_split for sk in joblst])
sw_skill_counts = Counter([sk for joblst in sw_skill_split for sk in joblst])

In [105]:
comm_skill_counts.keys()

dict_keys(['Management', 'Ability To Meet Deadlines', 'Influencing Skills', 'Tenacity', 'Writing', 'Prioritization', 'Innovation', 'Relationship Management', 'Consulting', 'Customer Service', 'Confident Communicator', 'Detail Oriented', 'Filing', 'Communications', 'Microsoft Excel', 'Analytical Skills', 'Leadership', 'Problem Solving', 'Budgeting', 'Rapport Building', 'Sales', 'Multitasking', 'Microsoft Outlook', 'Packaging And Labeling', 'Advising', 'Planning', 'Governance', 'Operations', 'Training And Development', 'Virtual Teams', 'Microsoft Word', 'Microsoft Office', 'Mathematics', 'Teamwork', 'Record Keeping', 'Self-Awareness', 'Administrative Functions', 'Tactfulness', 'Professionalism', 'Persuasive Communication', 'Strong Work Ethic', 'Forecasting', 'Decision Making', 'Mentorship', 'Accountability', 'Proactivity', 'Research', 'Presentations', 'Information Technology', 'Investigation', 'Quality Assurance', 'Report Writing', 'Relationship Building', 'Italian Language', 'Time Manag

In [106]:
np.quantile(np.array(list(spec_skill_count.values())), [0.25, 0.5, 0.75, 0.95, 0.99])

array([  2.  ,   5.  ,  18.  , 155.  , 560.76])

In [107]:
q95_spec_skills_counts =[(k,w) for k,w in spec_skill_count.items() if w > 155]
q25_spec_skills_counts =[(k,w) for k,w in spec_skill_count.items() if w < 3]

In [115]:
q25_spec_skills_counts

[('Basic Safety Training', 2),
 ('Intraocular Lenses', 2),
 ('', 2),
 ('Risk Selection', 2),
 ('Next Unit Of Computing (NUC)', 2),
 ('Voltage Drop', 1),
 ('Nuclear Design', 1),
 ('Arc Flash', 1),
 ('Microsoft Office Live Meeting', 1),
 ('Bomgar', 1),
 ('Industrial Networking', 2),
 ('Craniofacial Surgery', 1),
 ('Derivative Valuation', 2),
 ('Software Engineering Process', 2),
 ('Documentum', 2),
 ('Computational Mathematics', 2),
 ('Redmine', 2),
 ('Cortex M', 1),
 ('Current Probes', 2),
 ('International Accounting Standards Board', 2),
 ('OpenAPI Specification', 1),
 ('Cloudhub', 2),
 ('Cloud Security Strategy', 1),
 ('HP Storageworks', 1),
 ('Skid Steer Loaders', 2),
 ('Herbaceous Plants', 1),
 ('Applications Of Artificial Intelligence', 2),
 ('Reservoir Modeling', 2),
 ('MockK', 2),
 ('Crashlytics', 1),
 ('Okhttp', 1),
 ('Near Field Communication', 2),
 ('Karate (Software)', 2),
 ('QuerySurge', 1),
 ('Kinaxis (Software)', 2),
 ('Supply Chain Collaboration', 1),
 ('Business Interact

In [112]:
q99_spec_skills_counts =[(k,w) for k,w in spec_skill_count.items() if w > 561]


In [114]:
q99_spec_skills_counts

[('Procurement', 2171),
 ('Business Development', 2633),
 ('Help Desk Support', 587),
 ('Auditing', 4256),
 ('Invoicing', 2701),
 ('Billing', 566),
 ('SAP Applications', 1092),
 ('Business Requirements', 1019),
 ('Customer Relationship Management', 2208),
 ('General Data Protection Regulation (GDPR)', 798),
 ('Pharmaceuticals', 916),
 ('Key Performance Indicators (KPIs)', 3414),
 ('Sales Prospecting', 1984),
 ('Supply Chain', 1415),
 ('Business Strategies', 577),
 ('Finance', 6043),
 ('Booking (Sales)', 1166),
 ('Quality Management', 603),
 ('Agile Methodology', 3361),
 ('Scrum (Software Development)', 1033),
 ('Business Support Systems', 648),
 ('Surgery', 656),
 ('Data Entry', 863),
 ('Construction', 1107),
 ('Microsoft Azure', 1442),
 ('SQL (Programming Language)', 2321),
 ('Data Analysis', 1995),
 ('Warehousing', 1915),
 ('Reconciliation', 1044),
 ('Accounting', 4732),
 ('Project Management', 1656),
 ('Investments', 767),
 ('Housekeeping', 1046),
 ('Personal Care', 983),
 ('Social 